# ✅ Face Mask Detection – Auto Path Google Colab Notebook
This notebook automatically detects train/val paths even if ZIP contains extra folders.


In [ ]:
import os
from google.colab import files
import zipfile

print('📢 Upload your dataset ZIP file (with train/ and val/ folders inside)…')
uploaded = files.upload()

zip_name = list(uploaded.keys())[0]
print('📦 Uploaded:', zip_name)

with zipfile.ZipFile(zip_name, 'r') as z:
    z.extractall('/content/dataset')

print('✅ ZIP Extracted! Searching folders...')

root_path = '/content/dataset'
train_path = None
val_path = None

for root, dirs, files in os.walk(root_path):
    if 'train' in dirs and 'val' in dirs:
        train_path = os.path.join(root, 'train')
        val_path = os.path.join(root, 'val')
        break

print('📁 Train Path:', train_path)
print('📁 Val Path:', val_path)

if train_path is None or val_path is None:
    raise Exception('❌ train/ or val/ folder not found. Fix ZIP structure!')
else:
    print('✅ Correct paths detected! Continue running next cells.')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = 224
BATCH_SIZE = 32

train_gen = ImageDataGenerator(rescale=1/255., rotation_range=20, zoom_range=0.2, horizontal_flip=True)
val_gen = ImageDataGenerator(rescale=1/255.)

train_data = train_gen.flow_from_directory(train_path, target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, class_mode='binary')
val_data = val_gen.flow_from_directory(val_path, target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, class_mode='binary')

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

base = MobileNetV2(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))
base.trainable = False

x = GlobalAveragePooling2D()(base.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base.input, outputs=output)
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(train_data, validation_data=val_data, epochs=10)
print('🎉 Training Completed!')

In [ ]:
model.save('/content/face_mask_model.h5')
print('💾 Model Saved Successfully!')